# 1.3: Building a language model

`n-gram`에 대한 배경 지식이 필요하시면, Stanford에서 만든 기본 개념이 잘 설명된 슬라이드가 있습니다. `resource_files/resources/language_modeling.pdf`에서 찾으실 수 있습니다. 

`kaldi`는 `ARPA` 포맷으로 `language model`을 저장합니다. 해당 포맷에 대한 자세한 설명은 [이 곳](https://cmusphinx.github.io/wiki/arpaformat/)에서 확인하실 수 있습니다. 

`ARPA` 포맷으로 결과물을 제공하는 널리 쓰이는 오픈 소스 언어 모델 툴은 `IRSTLM`입니다. `IRSTLM`의 매뉴얼은 `resource_files/resources/irstlm-manual.pdf`에서 확인하실 수 있습니다. 

이번 워크샵에서는 toy corpus를 바탕으로 `IRSTLM`을 이용한 언어 모델을 만들고 그 구조를 살펴보도록 하겠습니다. 

## the toy corpus

toy corpus는 `resource_files/language_model/animal_corpus.txt`에 있습니다. 이 toy corpus에서 한 줄에는 하나의 문장이 있으며, 문장 부호는 존재하지 않습니다. 

**Note:** Language model 구축에 있어서, 문장 부호를 중요하게 여긴다면 문장 부호 또한 language model에 포함시킬 수 있습니다. 하지만 이 워크샵에서는 *spoken data*에 대한 language model 구축이 목적이므로, 문장 부호를 제외하도록 하겠습니다. 

In [ ]:
cat resource_files/language_model/animal_corpus.txt

## building the language model with `IRSTLM`

`IRSTLM`을 편리하게 사용하기 위해서 두 개의 `variable`을 미리 설정하도록 하겠습니다. 

In [ ]:
export IRSTLM=${KALDI_PATH}/tools/irstlm
export PATH=${PATH}:${IRSTLM}/bin

### `add-start-end.sh`

현재 corpus에는 `마침표 (.)`가 없으므로, 문장의 *시작*과 *끝*을 나타내는 부호를 추가해야 합니다. 

In [ ]:
add-start-end.sh -h

In [ ]:
cat resource_files/language_model/animal_corpus.txt | add-start-end.sh > resource_files/language_model/animal_corpus_start_stop.txt

In [ ]:
cat resource_files/language_model/animal_corpus_start_stop.txt

### `build-lm.sh`

Now let's build the actual language model using `build-lm.sh`

이제 `build-lm.sh` 파일을 이용하여 실제 language model을 구축해보겠습니다.  

In [ ]:
build-lm.sh -h

`build-lm.sh`에서 중요한 논항은 다음과 같습니다. 

 - `-i`
 - `-o`
 - `-n`

`-k`는 매우 큰 데이터에서 효율적으로 language model을 구축할 때 중요하게 사용되는 flag입니다. 현재 우리는 아주 작은 toy corpus를 사용하므로, `-k` flag는 신경을 쓰지 않아도 좋습니다. `-s`는 여러가지 smoothing 옵션을 제공합니다. `Smoothing`에 대한 자료는 Stanford에서 잘 정리하여 둔 것이 있습니다. `resource_files/smoothing_explained.pdf`에서 확인하실 수 있습니다. 이 toy corpus에서는 `-k`와 `-s` flag는 다루지 않겠습니다. 

In [ ]:
build-lm.sh \
    -i resource_files/language_model/animal_corpus_start_stop.txt \
    -o resource_files/language_model/animal_lm-2_gram.iarpa \
    -n 2

`IRSTLM`은 자동적으로 생성된 `language model`을 압축합니다. 내부를 살펴보기 위해서는 압축을 풀어주어야 합니다. 

In [ ]:
gzip -d resource_files/language_model/animal_lm-2_gram.iarpa.gz

In [ ]:
cat resource_files/language_model/animal_lm-2_gram.iarpa

## `iARPA` to `ARPA` format


생성된 language model의 헤더에는 `iARPA`라고 명시되어 있습니다. `IRSTLM` 매뉴얼은 해당 내용을 다음과 같이 설명합니다.

```
Notice that build-lm.sh produces a LM file train.ilm.gz that is NOT in the final ARPA format, but in an intermediate format called iARPA, that is recognized by the compile-lm command and by the Moses SMT decoder running with IRSTLM. 
```

`iARPA`와 `ARPA`의 차이는 다음과 같습니다.

```
This is an intermediate ARPA format in the sense that each entry of the file does not contain in the first position the full n-gram probability, but just its smoothed frequency.
```

그러므로 생성된 `iARPA` 파일들을 `compile-lm`을 이용하여 `kaldi`에서 요구하는 `ARPA` 포맷으로 만들어 주어야 합니다.

In [ ]:
compile-lm

In [ ]:
compile-lm resource_files/language_model/animal_lm-2_gram.iarpa --text=yes resource_files/language_model/animal_lm-2_gram.arpa


`ARPA`와 `iARPA`의 차이는 다음과 같이 확인할 수 있습니다.

In [ ]:
diff resource_files/language_model/animal_lm-2_gram.arpa resource_files/language_model/animal_lm-2_gram.iarpa

Now let's build a `2-gram` language model that does **not** include `start` and `stop` symbols.  We can do this by using our original `animal_corpus.txt` file as `input`.

**Note:** We can run `compile-lm` over the `gz`ed output of `build-lm.sh`, so we can skip the manual step of `decompress`ing the `iarpa.gz` file.

이제 `start`와 `stop` 기호를 제외한 corpus를 이용하여 `2-gram` language model을 만들어 보겠습니다. 이 language model은 기존 `animal_corpus.txt` 파일을 사용하여 만들어 볼 수 있습니다. 

In [ ]:
build-lm.sh \
    -i resource_files/language_model/animal_corpus.txt \
    -o resource_files/language_model/animal_lm-2_gram-no_start_stop.iarpa \
    -n 2
    
compile-lm \
    resource_files/language_model/animal_lm-2_gram-no_start_stop.iarpa.gz \
    --text=yes \
    resource_files/language_model/animal_lm-2_gram-no_start_stop.arpa


In [ ]:
cat resource_files/language_model/animal_lm-2_gram-no_start_stop.arpa

Let's also build a `3-gram` and a `4-gram` model, both using `start` and `stop` symbols.

이번에는 `start`와 `stop` 기호를 포함한 데이터를 이용하여 `3-gram`과 `4-gram` 모델을 만들어 보겠습니다.

In [ ]:
for n in 3 4; do
    lm_out=resource_files/language_model/animal_lm-${n}_gram
    
    # build the `iarpa` format
    build-lm.sh \
        -i resource_files/language_model/animal_corpus_start_stop.txt \
        -o ${lm_out}.iarpa \
        -n ${n}

    # compile into `arpa` format
    compile-lm \
        ${lm_out}.iarpa.gz \
        --text=yes \
        ${lm_out}.arpa

    # decompress `iarpa` format
    gzip -d ${lm_out}.iarpa.gz
done

We'll just `decompress` the remaining `gzip`ped file, and then we should have **four** language models, in both the `ARPA` and the `iARPA` formats.

남은 `gzip` 파일들도 압축을 풀어주면 총 4 개의 language model이 있음을 확인할 수 있습니다. 각각의 language model은 `ARPA`와 `iARPA` 포맷의 파일들을 모두 가지고 있습니다. 

In [ ]:
gzip -d resource_files/language_model/*.gz

In [ ]:
ls resource_files/language_model | grep "\.iarpa"

In [ ]:
ls resource_files/language_model | grep "\.arpa"

`ASR` pipeline에서는 `ARPA` 포맷이 필요합니다. 하지만 `python` 패키지에서는 `iARPA` 포맷을 이용하여서 language model의 내부를 확인합니다. 

`python` 패키지는 포맷에 너무 민감하여 `iARPA` 파일을 그 자체로 불러오지 못합니다. 그러므로 `sed` 명령어(찾아바꾸기)를 이용하여서 `python`에서 `.iarpa` 파일을 읽을 수 있도록 하겠습니다. 

In [ ]:
for lm in `ls resource_files/language_model/*.iarpa`; do
    sed -i.bak -E "s:([\-\.0-9]+) :\1\t:g" ${lm}
    rm resource_files/language_model/*.bak
done